# Day 9: Disk Fragmenter

Another push of the button leaves you in the familiar hallways of some friendly amphipods! Good thing you each somehow got your own personal mini submarine. The Historians jet away in search of the Chief, mostly by driving directly into walls.

While The Historians quickly figure out how to pilot these things, you notice an amphipod in the corner struggling with his computer. He's trying to make more contiguous free space by compacting all of the files, but his program isn't working; you offer to help.

He shows you the disk map (your puzzle input) he's already generated. For example:

```
2333133121414131402
```

The disk map uses a dense format to represent the layout of files and free space on the disk. The digits alternate between indicating the length of a file and the length of free space.

So, a disk map like `12345` would represent a one-block file, two blocks of free space, a three-block file, four blocks of free space, and then a five-block file. A disk map like `90909` would represent three nine-block files in a row (with no free space between them).

Each file on disk also has an ID number based on the order of the files as they appear before they are rearranged, starting with ID 0. So, the disk map `12345` has three files: a one-block file with ID 0, a three-block file with ID 1, and a five-block file with ID 2. Using one character for each block where digits are the file ID and `.` is free space, the disk map `12345` represents these individual blocks:

```
0..111....22222
```

The first example above, `2333133121414131402`, represents these individual blocks:

```
00...111...2...333.44.5555.6666.777.888899
```

The amphipod would like to move file blocks one at a time from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks). For the disk map `12345`, the process looks like this:

```
0..111....22222
02.111....2222.
022111....222..
0221112...22...
02211122..2....
022111222......
```

The first example requires a few more steps:

```
00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
```

The final step of this file-compacting process is to update the filesystem checksum. To calculate the checksum, add up the result of multiplying each of these blocks' position with the file ID number it contains. The leftmost block is in position 0. If a block contains free space, skip it instead.

Continuing the first example, the first few blocks' position multiplied by its file ID number are:

- `0 * 0 = 0`
- `1 * 0 = 0`
- `2 * 9 = 18`
- `3 * 9 = 27`
- `4 * 8 = 32`

... and so on. In this example, the checksum is the sum of these, `1928`.

---

## Task

Compact the amphipod's hard drive using the process he requested. What is the resulting filesystem checksum?

In [1]:
import itertools


class Disk:
    def __init__(self, filepath: str):
        self.filepath = filepath
        self.disk_map = self._read_disk_map()
        self.blocks = self._get_blocks()

    def _read_disk_map(self) -> list[str]:
        """Reads the disk map from the file."""
        with open(self.filepath, "r", encoding="utf-8") as f:
            return [list(line.strip()) for line in f][0]

    @staticmethod
    def _is_odd(x: int) -> bool:
        """Checks if a number is odd."""
        return x & 1 == 0

    @staticmethod
    def _is_free(block: int) -> bool:
        """Checks if a block is free (negative value)."""
        return block < 0

    @staticmethod
    def _repeat_n_times(a: int, n: int) -> list[int]:
        """Repeats a value `a` `n` times."""
        return [a] * n

    def _get_blocks(self) -> list[int]:
        """Converts the disk map into a list of blocks."""
        block_groups = [
            self._repeat_n_times(-1 if self._is_odd(i + 1) else i // 2, int(s))
            for i, s in enumerate(self.disk_map, start=0)
        ]
        return list(itertools.chain.from_iterable(block_groups))

    def move_single_files(self, verbose: bool = False) -> None:
        """Moves file blocks one at a time from end to leftmost free space"""
        file_idx = len(self.blocks) - 1
        i = 0

        while file_idx > i:
            if self._is_free(self.blocks[file_idx]):
                file_idx -= 1
                continue
            if self._is_free(self.blocks[i]):
                # simultaneous assignment
                self.blocks[i], self.blocks[file_idx] = (
                    self.blocks[file_idx],
                    self.blocks[i],
                )
                if verbose:
                    self.print_blocks()
            i += 1

    def checksum(self) -> int:
        """Calculates the checksum of the disk."""
        return sum(
            i * block for i, block in enumerate(self.blocks) if not self._is_free(block)
        )

    def print_blocks(self, blocks: list[int] = None) -> None:
        """Prints the disk blocks as a visual representation."""
        if not blocks:
            blocks = self.blocks
        print("".join("." if self._is_free(block) else str(block) for block in blocks))


# Usage
disk = Disk("./example.txt")
disk.print_blocks()
disk.move_single_files(verbose=True)
disk.print_blocks()
print("Checksum:", disk.checksum())

00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
0099811188827773336446555566..............
Checksum: 1928


In [2]:
disk = Disk("./input.txt")
disk.move_single_files(verbose=False)
print("Checksum:", disk.checksum())

Checksum: 6398608069280


# Part Two

Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number, starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

The first example from above now proceeds differently:

```
00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
```

The process of updating the filesystem checksum is the same; now, this example's checksum would be `2858`.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?

In [3]:
from tqdm.auto import tqdm


class Disk2(Disk):
    @staticmethod
    def _is_free_group(blocks: int) -> bool:
        """Checks if a block is free (negative value)."""
        return all(block < 0 for block in blocks)

    def move_whole_files(self, verbose: bool = False):
        for _id in tqdm(range(max(self.blocks), -1, -1), desc="Processing ID"):
            groups = [list(group) for _, group in itertools.groupby(self.blocks)]
            target_idx = next((i for i, g in enumerate(groups) if _id in g), None)

            for i, group in enumerate(groups):
                if i >= target_idx:
                    break  # free space only to the left
                if not self._is_free_group(group):
                    continue
                len_free, len_target = len(group), len(groups[target_idx])
                if len_target > len_free:  # whole block does NOT fit in free block
                    continue

                group[:len_target], groups[target_idx] = (
                    groups[target_idx],
                    group[:len_target],
                )  # simultaneous assignment

                # update blocks
                self.blocks = list(itertools.chain.from_iterable(groups))
                if verbose:
                    self.print_blocks()
                break


disk = Disk2("./example.txt")
disk.move_whole_files(verbose=True)
print("Checksum:", disk.checksum())

Processing ID:   0%|          | 0/10 [00:00<?, ?it/s]

0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
Checksum: 2858


In [4]:
disk = Disk2("./input.txt")
disk.move_whole_files(verbose=False)
print("Checksum:", disk.checksum())

Processing ID:   0%|          | 0/10000 [00:00<?, ?it/s]

Checksum: 6427437134372
